In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import shap
import dalex as dx
from eli5.sklearn import PermutationImportance
from yellowbrick.model_selection import FeatureImportances 
import eli5

Using TensorFlow backend.


In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso 
from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import LinearSVR

from sklearn.ensemble import VotingRegressor

In [4]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

In [5]:
from sklearn.metrics import (mean_squared_error, 
                             mean_absolute_error, 
                             r2_score, 
                             mean_squared_log_error)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [7]:
dataset = pd.read_csv("Data/neonate_dataset.csv")

In [8]:
dataset.fillna(method="ffill", inplace=True)
y = dataset['LOS']
X = dataset.drop(['LOS'], axis=1)

In [9]:
def cv_partitioning(df, k, fraction): 
    dataset_list = [] 
    
    for i in range(0, k): 
        dataset_list.append(df.sample(frac=fraction))
    
    return dataset_list

In [10]:
def get_results(train, test, regressors): 
    
    train_x = train.drop(["LOS"], axis=1) 
    train_y = train["LOS"] 
    
    test_x = test.drop(["LOS"], axis=1) 
    test_y = test["LOS"]
    
    reg_names = [] 
    mse_scores = [] 
    mae_scores = [] 
    r2_scores = []    
    
    for reg in regressors:  
        reg.fit(train_x, train_y)
        y_preds = reg.predict(test_x)
        
        reg_names.append(reg.__class__.__name__)
        mse_scores.append(mean_squared_error(test_y, y_preds, squared=False))
        mae_scores.append(mean_absolute_error(test_y, y_preds))
        r2_scores.append(r2_score(test_y, y_preds))
        
    
    results = { 'name': reg_names, 
                'rmse': mse_scores, 
                'mae': mae_scores, 
                'r2': r2_scores, 
              }
       
    df = pd.DataFrame.from_dict(results)
    
    return df 

In [11]:
selected_features_classification = [
     'temperature_mean',
     'respRate_std',
     'skinTemperature_std',
     'skinTemperature_var',
     'heartRate_std',
     'hrAlarmLow_max',
     'hrAlarmLow_std',
     'hrAlarmHigh_std', 
     'sao2_std',
     'heartRate_mean',
     'respRate_mean',
     'skinTemperature_mean',
     'bpCuffDiastolic_std',
     'BIRTH_WEIGHT',
     'bpCuffSystolic_mean',
     'bpCuffDiastolic_mean',
     'sao2_mean',
     'temperature_std',
     'bpCuffMean_mean',
     'PLATELET',
     'D10W_MEAN', 'ADMISSION_LOCATION', 'INSURANCE', 'GESTATION', 'WHITE_BLOOD', 
     'DATEEVENTS', 'DW10_COUNT', 'PRESCRIPTIONS', 'HEAD_CIRC'] 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [13]:
train_data = X_train[selected_features_classification]
train_data["LOS"] = y_train 

test_data = X_test[selected_features_classification]
test_data["LOS"] = y_test

train_dfs = cv_partitioning(train_data, 10, 0.95)

In [16]:
regressors = [KNeighborsRegressor(n_neighbors=7, leaf_size=30, p=1, algorithm='auto'), 
              LinearRegression(), 
              Ridge(alpha=3), 
              Lasso(alpha=0.1), 
              LinearSVR(C=0.009), 
              DecisionTreeRegressor(criterion='mse', max_depth=4)]

results_data = []
for i in range(0, 10): 
    results_data.append(get_results(train_dfs[i], test_data, regressors))
    
resultsDF = pd.concat(results_data)

In [18]:
names = ['KNeighborsRegressor', 'LinearRegression', 'LinearSVR', 'DecisionTreeRegressor', 'Ridge', 'Lasso']

rmse_list = [] 
rmse_std = [] 

mae_list = []
mae_std = [] 

r2_list = []
r2_std = [] 

for n in names: 
    rmse_list.append(resultsDF[resultsDF.name == n].rmse.mean())
    rmse_std.append(resultsDF[resultsDF.name == n].rmse.std()*10)
    mae_list.append(resultsDF[resultsDF.name == n].mae.mean())
    mae_std.append(resultsDF[resultsDF.name == n].mae.std()*10)
    r2_list.append(resultsDF[resultsDF.name == n].r2.mean()*100)   
    r2_std.append(resultsDF[resultsDF.name == n].r2.std()*100)

final_results = {"method": names, 
                "rmse": rmse_list, 
                "rmse_std": rmse_std, 
                "mae": mae_list, 
                "mae_std": mae_std, 
                "r2": r2_list, 
                "r2_std": r2_std}

final_resultsDF = pd.DataFrame.from_dict(final_results)

final_resultsDF

,method,rmse,rmse_std,mae,mae_std,r2,r2_std
0,KNeighborsRegressor,18.145084,1.214687,11.753438,0.880087,53.613576,0.620948
1,LinearRegression,14.814718,0.291188,10.823606,0.244075,69.079700,0.121574
2,LinearSVR,16.027561,0.777351,10.832164,0.685451,63.809093,0.351343
3,DecisionTreeRegressor,15.519825,6.818964,10.516668,2.624863,66.007516,3.033045
4,Ridge,14.818913,0.295444,10.822710,0.249650,69.062183,0.123391
5,Lasso,14.861171,0.274189,10.808947,0.231675,68.885502,0.114870


In [19]:
voting = VotingRegressor([('lr', LinearRegression()), 
                        ('ridge', Ridge(alpha=2.5, max_iter=1500)),
                        ('dt', DecisionTreeRegressor(criterion='friedman_mse', max_depth=4)), 
                        ])

In [20]:
results_data = []
for i in range(0, 1): 
    results_data.append(get_results(train_dfs[i], test_data, [voting]))

In [21]:
results_data

[              name       rmse        mae        r2
 0  VotingRegressor  13.977583  10.022176  0.724755]

## <center><font color='green'> Feature Importance 

In [22]:
def explainability(model, train, test): 
    train_x = train.drop(["LOS"], axis=1) 
    train_y = train["LOS"] 
    
    test_x = test.drop(["LOS"], axis=1) 
    test_y = test["LOS"]
    
    
    model.fit(train_x, train_y)
    # dalex 
    exp = dx.Explainer(model, test_x, test_y)
    exp.model_parts().plot()
    
    # Permutation Importance 
    perm = PermutationImportance(model, random_state=1).fit(test_x, test_y)
    fs_scores = eli5.show_weights(perm, feature_names = test_x.columns.tolist())

#     viz = FeatureImportances(model) 
#     viz.fit(test_x, test_y)
#     viz.show()
    
#     explainer = shap.TreeExplainer(model)
#     shap_values = explainer.shap_values(test_x)    
#     shap.summary_plot(shap_values, test_x, feature_names=test_x.columns)
    
    return fs_scores

In [23]:
regressors = [KNeighborsRegressor(n_neighbors=7, leaf_size=30, p=1, algorithm='auto'), 
              LinearRegression(), 
              Ridge(alpha=3), 
              Lasso(alpha=0.1), 
              LinearSVR(C=0.009), 
              DecisionTreeRegressor(criterion='mse', max_depth=4)]

In [24]:
voting = VotingRegressor([('lr', LinearRegression()), 
                        ('ridge', Ridge(alpha=2.5, max_iter=1500)),
                        ('dt', DecisionTreeRegressor(criterion='friedman_mse', max_depth=4)), 
                        ])

In [26]:
res = explainability(Lasso(alpha=0.1), 
                    train_dfs[0], test_data)
res

Preparation of a new explainer is initiated

  -> data              : 499 rows 29 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 499 values
  -> model_class       : sklearn.linear_model._coordinate_descent.Lasso (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x00000261233EEDC0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = -20.0, mean = 25.1, max = 1.07e+02
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -48.4, mean = -1.04, max = 89.1
  -> model_info        : package sklearn

A new explainer has been created!


Weight,Feature
0.2385 ± 0.0332,HEAD_CIRC
0.0768 ± 0.0264,GESTATION
0.0696 ± 0.0186,PRESCRIPTIONS
0.0368 ± 0.0132,BIRTH_WEIGHT
0.0273 ± 0.0087,DATEEVENTS
0.0271 ± 0.0139,sao2_mean
0.0103 ± 0.0029,respRate_std
0.0090 ± 0.0031,WHITE_BLOOD
0.0081 ± 0.0090,bpCuffDiastolic_mean
0.0077 ± 0.0053,heartRate_mean


## <center><font color='green'> Feature Importance - Static Regressor 

In [59]:
selected_features_classification = [
     'temperature_mean',
     'respRate_std',
     'skinTemperature_std',
     'skinTemperature_var',
     'sao2_std',
     'heartRate_mean',
     'respRate_mean',
     'skinTemperature_mean',
     'bpCuffDiastolic_std',
     'BIRTH_WEIGHT',
     'bpCuffSystolic_mean',
     'bpCuffDiastolic_mean',
     'sao2_mean',
     'temperature_std',
     'temperature_var',
     'PLATELET',
     'D10W_MEAN', 'ADMISSION_LOCATION', 'INSURANCE', 'GESTATION', 'WHITE_BLOOD', 
     'DATEEVENTS', 'DW10_COUNT', 'PRESCRIPTIONS', 'HEAD_CIRC'] 

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=25)

In [61]:
X_train_fs = X_train[selected_features_classification]
X_test_fs = X_test[selected_features_classification]

train_data = X_train_fs[:]
train_data["LOS"] = y_train 

test_data = X_test_fs[:]
test_data["LOS"] = y_test

train_dfs = cv_partitioning(train_data, 10, 0.98)

In [44]:
regressors = [RandomForestRegressor(n_estimators = 200, max_depth = 8), 
              ExtraTreesRegressor(n_estimators = 200, max_depth = 10), 
              XGBRegressor(max_depth=3, n_estimators=120, n_jobs=-1, objectvie='reg:r2', booster='gbtree', random_state=25, learning_rate=0.05, gamma=5), 
              LGBMRegressor(n_estimators=200, max_depth=3, learning_rate=0.05), 
              GradientBoostingRegressor(n_estimators = 150, max_depth = 3, learning_rate=0.05), 
              CatBoostRegressor(learning_rate=0.03, depth=5), 
              AdaBoostRegressor(n_estimators=80, learning_rate=0.05)]

results_data = []
for i in range(0, 10): 
    results_data.append(get_results(train_dfs[i], test_data, regressors))

[11:43:40] WARNING: ..\src\learner.cc:541: 
Parameters: { objectvie } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0:	learn: 26.4803599	total: 143ms	remaining: 2m 22s
1:	learn: 26.0175362	total: 144ms	remaining: 1m 11s
2:	learn: 25.6142148	total: 145ms	remaining: 48.3s
3:	learn: 25.1594848	total: 146ms	remaining: 36.5s
4:	learn: 24.7317850	total: 148ms	remaining: 29.4s
5:	learn: 24.3371820	total: 149ms	remaining: 24.6s
6:	learn: 23.9670811	total: 150ms	remaining: 21.3s
7:	learn: 23.6126318	total: 151ms	remaining: 18.8s
8:	learn: 23.2512290	total: 153ms	remaining: 16.8s
9:	learn: 22.8911528	total: 154ms	remaining: 15.2s
10:	learn: 22.5290256	total: 155ms	remaining: 14s
11:	learn: 22.1990614	total: 156ms	remaining: 12.9s
12:	learn: 21.8784097	total: 158ms	remaining: 12s
13:	lea

282:	learn: 10.8350373	total: 476ms	remaining: 1.21s
283:	learn: 10.8266156	total: 477ms	remaining: 1.2s
284:	learn: 10.8222065	total: 479ms	remaining: 1.2s
285:	learn: 10.8124532	total: 480ms	remaining: 1.2s
286:	learn: 10.8085075	total: 481ms	remaining: 1.19s
287:	learn: 10.8016296	total: 482ms	remaining: 1.19s
288:	learn: 10.7966176	total: 483ms	remaining: 1.19s
289:	learn: 10.7896590	total: 484ms	remaining: 1.19s
290:	learn: 10.7779842	total: 485ms	remaining: 1.18s
291:	learn: 10.7704877	total: 487ms	remaining: 1.18s
292:	learn: 10.7656218	total: 488ms	remaining: 1.18s
293:	learn: 10.7601837	total: 489ms	remaining: 1.17s
294:	learn: 10.7518624	total: 491ms	remaining: 1.17s
295:	learn: 10.7421786	total: 492ms	remaining: 1.17s
296:	learn: 10.7351060	total: 493ms	remaining: 1.17s
297:	learn: 10.7214497	total: 494ms	remaining: 1.16s
298:	learn: 10.7161929	total: 496ms	remaining: 1.16s
299:	learn: 10.7105327	total: 497ms	remaining: 1.16s
300:	learn: 10.6977246	total: 498ms	remaining: 1.

535:	learn: 9.2262470	total: 810ms	remaining: 701ms
536:	learn: 9.2223412	total: 811ms	remaining: 699ms
537:	learn: 9.2173474	total: 812ms	remaining: 698ms
538:	learn: 9.2123590	total: 814ms	remaining: 696ms
539:	learn: 9.2081448	total: 815ms	remaining: 694ms
540:	learn: 9.2054944	total: 816ms	remaining: 692ms
541:	learn: 9.2045059	total: 817ms	remaining: 690ms
542:	learn: 9.1988995	total: 818ms	remaining: 689ms
543:	learn: 9.1886981	total: 820ms	remaining: 687ms
544:	learn: 9.1785038	total: 821ms	remaining: 685ms
545:	learn: 9.1709669	total: 822ms	remaining: 684ms
546:	learn: 9.1633298	total: 823ms	remaining: 682ms
547:	learn: 9.1548908	total: 825ms	remaining: 680ms
548:	learn: 9.1444155	total: 826ms	remaining: 678ms
549:	learn: 9.1356758	total: 827ms	remaining: 677ms
550:	learn: 9.1292627	total: 828ms	remaining: 675ms
551:	learn: 9.1250723	total: 829ms	remaining: 673ms
552:	learn: 9.1189655	total: 831ms	remaining: 671ms
553:	learn: 9.1157138	total: 832ms	remaining: 670ms
554:	learn: 

828:	learn: 7.8896533	total: 1.14s	remaining: 236ms
829:	learn: 7.8856365	total: 1.15s	remaining: 235ms
830:	learn: 7.8797862	total: 1.15s	remaining: 233ms
831:	learn: 7.8759131	total: 1.15s	remaining: 232ms
832:	learn: 7.8726735	total: 1.15s	remaining: 230ms
833:	learn: 7.8698290	total: 1.15s	remaining: 229ms
834:	learn: 7.8665199	total: 1.15s	remaining: 228ms
835:	learn: 7.8616652	total: 1.15s	remaining: 226ms
836:	learn: 7.8602447	total: 1.15s	remaining: 225ms
837:	learn: 7.8561512	total: 1.15s	remaining: 223ms
838:	learn: 7.8542025	total: 1.16s	remaining: 222ms
839:	learn: 7.8526460	total: 1.16s	remaining: 220ms
840:	learn: 7.8519075	total: 1.16s	remaining: 219ms
841:	learn: 7.8484069	total: 1.16s	remaining: 218ms
842:	learn: 7.8455140	total: 1.16s	remaining: 216ms
843:	learn: 7.8430520	total: 1.16s	remaining: 215ms
844:	learn: 7.8373688	total: 1.16s	remaining: 213ms
845:	learn: 7.8319102	total: 1.16s	remaining: 212ms
846:	learn: 7.8289214	total: 1.17s	remaining: 210ms
847:	learn: 

[11:43:46] WARNING: ..\src\learner.cc:541: 
Parameters: { objectvie } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0:	learn: 26.2395925	total: 1.51ms	remaining: 1.51s
1:	learn: 25.8051037	total: 2.89ms	remaining: 1.44s
2:	learn: 25.4045121	total: 4.08ms	remaining: 1.36s
3:	learn: 24.9575656	total: 5.28ms	remaining: 1.31s
4:	learn: 24.5550548	total: 6.56ms	remaining: 1.31s
5:	learn: 24.1676070	total: 7.97ms	remaining: 1.32s
6:	learn: 23.7989350	total: 9.86ms	remaining: 1.4s
7:	learn: 23.4526931	total: 11.4ms	remaining: 1.41s
8:	learn: 23.0966089	total: 12.6ms	remaining: 1.39s
9:	learn: 22.7797651	total: 13.8ms	remaining: 1.36s
10:	learn: 22.4417829	total: 14.9ms	remaining: 1.34s
11:	learn: 22.1090685	total: 16.1ms	remaining: 1.33s
12:	learn: 21.8047394	total: 17.3ms	remaining:

...

681:	learn: 8.5758968	total: 815ms	remaining: 380ms
682:	learn: 8.5704307	total: 816ms	remaining: 379ms
683:	learn: 8.5637533	total: 817ms	remaining: 378ms
684:	learn: 8.5570273	total: 819ms	remaining: 376ms
685:	learn: 8.5555689	total: 820ms	remaining: 375ms
686:	learn: 8.5484115	total: 821ms	remaining: 374ms
687:	learn: 8.5439855	total: 822ms	remaining: 373ms
688:	learn: 8.5406415	total: 823ms	remaining: 372ms
689:	learn: 8.5389696	total: 825ms	remaining: 370ms
690:	learn: 8.5315055	total: 826ms	remaining: 369ms
691:	learn: 8.5239648	total: 827ms	remaining: 368ms
692:	learn: 8.5165942	total: 828ms	remaining: 367ms
693:	learn: 8.5138871	total: 829ms	remaining: 366ms
694:	learn: 8.5076894	total: 831ms	remaining: 364ms
695:	learn: 8.4985850	total: 832ms	remaining: 363ms
696:	learn: 8.4910318	total: 833ms	remaining: 362ms
697:	learn: 8.4866492	total: 834ms	remaining: 361ms
698:	learn: 8.4812314	total: 835ms	remaining: 360ms
699:	learn: 8.4737344	total: 836ms	remaining: 358ms
700:	learn: 

958:	learn: 7.3798884	total: 1.14s	remaining: 48.9ms
959:	learn: 7.3759250	total: 1.14s	remaining: 47.7ms
960:	learn: 7.3744513	total: 1.15s	remaining: 46.5ms
961:	learn: 7.3692320	total: 1.15s	remaining: 45.3ms
962:	learn: 7.3678316	total: 1.15s	remaining: 44.1ms
963:	learn: 7.3639448	total: 1.15s	remaining: 42.9ms
964:	learn: 7.3610845	total: 1.15s	remaining: 41.7ms
965:	learn: 7.3569466	total: 1.15s	remaining: 40.5ms
966:	learn: 7.3521141	total: 1.15s	remaining: 39.3ms
967:	learn: 7.3477614	total: 1.15s	remaining: 38.1ms
968:	learn: 7.3462402	total: 1.15s	remaining: 36.9ms
969:	learn: 7.3429802	total: 1.16s	remaining: 35.8ms
970:	learn: 7.3390992	total: 1.16s	remaining: 34.6ms
971:	learn: 7.3369513	total: 1.16s	remaining: 33.4ms
972:	learn: 7.3352646	total: 1.16s	remaining: 32.2ms
973:	learn: 7.3340586	total: 1.16s	remaining: 31ms
974:	learn: 7.3306788	total: 1.16s	remaining: 29.8ms
975:	learn: 7.3284305	total: 1.16s	remaining: 28.6ms
976:	learn: 7.3271611	total: 1.16s	remaining: 27

138:	learn: 12.2474202	total: 163ms	remaining: 1.01s
139:	learn: 12.2247695	total: 165ms	remaining: 1.01s
140:	learn: 12.2029808	total: 166ms	remaining: 1.01s
141:	learn: 12.1848895	total: 167ms	remaining: 1.01s
142:	learn: 12.1780530	total: 168ms	remaining: 1.01s
143:	learn: 12.1668766	total: 169ms	remaining: 1.01s
144:	learn: 12.1549804	total: 171ms	remaining: 1.01s
145:	learn: 12.1500539	total: 172ms	remaining: 1s
146:	learn: 12.1410256	total: 173ms	remaining: 1s
147:	learn: 12.1291020	total: 174ms	remaining: 1s
148:	learn: 12.1151039	total: 176ms	remaining: 1s
149:	learn: 12.1051485	total: 177ms	remaining: 1s
150:	learn: 12.0907415	total: 178ms	remaining: 1s
151:	learn: 12.0740044	total: 179ms	remaining: 1s
152:	learn: 12.0653114	total: 180ms	remaining: 999ms
153:	learn: 12.0522014	total: 182ms	remaining: 998ms
154:	learn: 12.0364761	total: 183ms	remaining: 997ms
155:	learn: 12.0243252	total: 184ms	remaining: 995ms
156:	learn: 12.0106373	total: 185ms	remaining: 994ms
157:	learn: 12

427:	learn: 9.9265071	total: 493ms	remaining: 659ms
428:	learn: 9.9216080	total: 494ms	remaining: 658ms
429:	learn: 9.9152540	total: 495ms	remaining: 657ms
430:	learn: 9.9118411	total: 497ms	remaining: 655ms
431:	learn: 9.9072371	total: 498ms	remaining: 654ms
432:	learn: 9.8950327	total: 499ms	remaining: 653ms
433:	learn: 9.8852505	total: 500ms	remaining: 652ms
434:	learn: 9.8773919	total: 501ms	remaining: 651ms
435:	learn: 9.8767007	total: 502ms	remaining: 650ms
436:	learn: 9.8721429	total: 503ms	remaining: 649ms
437:	learn: 9.8700628	total: 505ms	remaining: 647ms
438:	learn: 9.8645399	total: 506ms	remaining: 646ms
439:	learn: 9.8522065	total: 507ms	remaining: 645ms
440:	learn: 9.8470957	total: 508ms	remaining: 644ms
441:	learn: 9.8411782	total: 509ms	remaining: 643ms
442:	learn: 9.8373033	total: 510ms	remaining: 642ms
443:	learn: 9.8329586	total: 511ms	remaining: 641ms
444:	learn: 9.8303259	total: 513ms	remaining: 639ms
445:	learn: 9.8295742	total: 514ms	remaining: 638ms
446:	learn: 

720:	learn: 8.5147491	total: 825ms	remaining: 319ms
721:	learn: 8.5063624	total: 827ms	remaining: 318ms
722:	learn: 8.5022701	total: 828ms	remaining: 317ms
723:	learn: 8.4989256	total: 829ms	remaining: 316ms
724:	learn: 8.4959152	total: 830ms	remaining: 315ms
725:	learn: 8.4898037	total: 831ms	remaining: 314ms
726:	learn: 8.4882849	total: 832ms	remaining: 313ms
727:	learn: 8.4820305	total: 833ms	remaining: 311ms
728:	learn: 8.4744561	total: 835ms	remaining: 310ms
729:	learn: 8.4713254	total: 836ms	remaining: 309ms
730:	learn: 8.4696722	total: 837ms	remaining: 308ms
731:	learn: 8.4658926	total: 838ms	remaining: 307ms
732:	learn: 8.4621231	total: 839ms	remaining: 306ms
733:	learn: 8.4578253	total: 840ms	remaining: 305ms
734:	learn: 8.4547104	total: 842ms	remaining: 303ms
735:	learn: 8.4497418	total: 843ms	remaining: 302ms
736:	learn: 8.4460573	total: 844ms	remaining: 301ms
737:	learn: 8.4435484	total: 845ms	remaining: 300ms
738:	learn: 8.4402339	total: 846ms	remaining: 299ms
739:	learn: 

In [46]:
final_resultsDF

,method,rmse,rmse_std,mae,mae_std,r2,r2_std
0,CatBoostRegressor,12.506511,0.073501,8.531761,0.061730,78.353646,0.254741
1,XGBRegressor,13.019346,0.084459,8.850942,0.052038,76.541852,0.304998
2,LGBMRegressor,13.057718,0.081572,8.889987,0.048274,76.403438,0.294485
3,GradientBoostingRegressor,12.989206,0.105422,8.842636,0.057622,76.649838,0.379409
4,RandomForestRegressor,13.294270,0.042333,8.893889,0.034379,75.541382,0.155514
5,ExtraTreesRegressor,12.888777,0.097216,8.765151,0.054459,77.009704,0.345772
6,AdaBoostRegressor,14.193935,0.084625,10.166886,0.056907,72.118352,0.330848


In [85]:
voting = VotingRegressor([('etr', ExtraTreesRegressor(n_estimators = 200, max_depth = 10)), 
#                         ('xgb', XGBRegressor(max_depth=3, n_estimators=120, n_jobs=-1,
#                            objectvie='reg:r2', booster='gbtree',
#                            random_state=25, learning_rate=0.05, gamma=5)), 
#                         ('lgb', LGBMRegressor(n_estimators=200, max_depth=3, learning_rate=0.05)),
                        ('gb', GradientBoostingRegressor(n_estimators=50, max_depth=4)), 
                        ('cat', CatBoostRegressor(learning_rate=0.03, depth=5))], 
                         weights=[0.5, 0.5, 6.0]
                        ) 

In [86]:
results_data = []
for i in range(0, 1): 
    results_data.append(get_results(train_dfs[i], test_data, [voting]))

0:	learn: 26.4399176	total: 1.41ms	remaining: 1.41s
1:	learn: 25.9929053	total: 2.85ms	remaining: 1.42s
2:	learn: 25.5786071	total: 4.02ms	remaining: 1.34s
3:	learn: 25.1214411	total: 5.15ms	remaining: 1.28s
4:	learn: 24.7176590	total: 6.33ms	remaining: 1.26s
5:	learn: 24.3163583	total: 7.46ms	remaining: 1.24s
6:	learn: 23.9467923	total: 8.58ms	remaining: 1.22s
7:	learn: 23.5863201	total: 9.89ms	remaining: 1.23s
8:	learn: 23.2187733	total: 11.1ms	remaining: 1.22s
9:	learn: 22.8661114	total: 12.2ms	remaining: 1.21s
10:	learn: 22.5167448	total: 13.3ms	remaining: 1.2s
11:	learn: 22.1771218	total: 14.5ms	remaining: 1.19s
12:	learn: 21.8530635	total: 15.6ms	remaining: 1.19s
13:	learn: 21.5634146	total: 16.7ms	remaining: 1.18s
14:	learn: 21.3064306	total: 17.8ms	remaining: 1.17s
15:	learn: 21.0415641	total: 19ms	remaining: 1.17s
16:	learn: 20.7576228	total: 20.2ms	remaining: 1.17s
17:	learn: 20.4794232	total: 21.4ms	remaining: 1.16s
18:	learn: 20.2158995	total: 22.5ms	remaining: 1.16s
19:	le

281:	learn: 10.7473359	total: 322ms	remaining: 820ms
282:	learn: 10.7386375	total: 323ms	remaining: 819ms
283:	learn: 10.7330872	total: 324ms	remaining: 818ms
284:	learn: 10.7194952	total: 325ms	remaining: 816ms
285:	learn: 10.7136313	total: 327ms	remaining: 815ms
286:	learn: 10.7022186	total: 328ms	remaining: 814ms
287:	learn: 10.6975037	total: 329ms	remaining: 813ms
288:	learn: 10.6923328	total: 330ms	remaining: 812ms
289:	learn: 10.6868167	total: 331ms	remaining: 811ms
290:	learn: 10.6790198	total: 332ms	remaining: 809ms
291:	learn: 10.6738387	total: 333ms	remaining: 808ms
292:	learn: 10.6688541	total: 334ms	remaining: 807ms
293:	learn: 10.6597896	total: 336ms	remaining: 806ms
294:	learn: 10.6523523	total: 337ms	remaining: 805ms
295:	learn: 10.6459387	total: 338ms	remaining: 804ms
296:	learn: 10.6416726	total: 339ms	remaining: 802ms
297:	learn: 10.6328219	total: 340ms	remaining: 801ms
298:	learn: 10.6248989	total: 341ms	remaining: 800ms
299:	learn: 10.6166219	total: 343ms	remaining:

577:	learn: 9.0243447	total: 653ms	remaining: 477ms
578:	learn: 9.0198752	total: 655ms	remaining: 476ms
579:	learn: 9.0132914	total: 656ms	remaining: 475ms
580:	learn: 9.0102610	total: 657ms	remaining: 474ms
581:	learn: 9.0053918	total: 658ms	remaining: 473ms
582:	learn: 9.0012341	total: 659ms	remaining: 471ms
583:	learn: 8.9985981	total: 660ms	remaining: 470ms
584:	learn: 8.9961008	total: 661ms	remaining: 469ms
585:	learn: 8.9881065	total: 662ms	remaining: 468ms
586:	learn: 8.9841619	total: 663ms	remaining: 467ms
587:	learn: 8.9786274	total: 665ms	remaining: 466ms
588:	learn: 8.9742629	total: 666ms	remaining: 464ms
589:	learn: 8.9711274	total: 667ms	remaining: 463ms
590:	learn: 8.9663510	total: 668ms	remaining: 462ms
591:	learn: 8.9623730	total: 669ms	remaining: 461ms
592:	learn: 8.9560000	total: 670ms	remaining: 460ms
593:	learn: 8.9497063	total: 672ms	remaining: 459ms
594:	learn: 8.9460287	total: 673ms	remaining: 458ms
595:	learn: 8.9429217	total: 674ms	remaining: 457ms
596:	learn: 

851:	learn: 7.8312859	total: 979ms	remaining: 170ms
852:	learn: 7.8295582	total: 980ms	remaining: 169ms
853:	learn: 7.8267523	total: 981ms	remaining: 168ms
854:	learn: 7.8236711	total: 982ms	remaining: 167ms
855:	learn: 7.8183155	total: 983ms	remaining: 165ms
856:	learn: 7.8151697	total: 984ms	remaining: 164ms
857:	learn: 7.8139390	total: 986ms	remaining: 163ms
858:	learn: 7.8110300	total: 987ms	remaining: 162ms
859:	learn: 7.8065270	total: 988ms	remaining: 161ms
860:	learn: 7.7995956	total: 989ms	remaining: 160ms
861:	learn: 7.7954970	total: 991ms	remaining: 159ms
862:	learn: 7.7903574	total: 992ms	remaining: 157ms
863:	learn: 7.7898204	total: 993ms	remaining: 156ms
864:	learn: 7.7862682	total: 994ms	remaining: 155ms
865:	learn: 7.7810924	total: 995ms	remaining: 154ms
866:	learn: 7.7766523	total: 997ms	remaining: 153ms
867:	learn: 7.7713277	total: 998ms	remaining: 152ms
868:	learn: 7.7690803	total: 999ms	remaining: 151ms
869:	learn: 7.7650158	total: 1s	remaining: 150ms
870:	learn: 7.7

In [87]:
results_data

[              name       rmse       mae        r2
 0  VotingRegressor  12.533312  8.502983  0.782614]

### Feature Importance 

In [91]:
def explainability(model, train, test): 
    train_x = train.drop(["LOS"], axis=1) 
    train_y = train["LOS"] 
    
    test_x = test.drop(["LOS"], axis=1) 
    test_y = test["LOS"]
    
    
    model.fit(train_x, train_y)
    # dalex 
    exp = dx.Explainer(model, test_x, test_y)
    exp.model_parts().plot()
    
    # Permutation Importance 
    perm = PermutationImportance(model, random_state=1).fit(test_x, test_y)
    fs_scores = eli5.show_weights(perm, feature_names = test_x.columns.tolist())

#     viz = FeatureImportances(model) 
#     viz.fit(test_x, test_y)
#     viz.show()
    
#     explainer = shap.TreeExplainer(model)
#     shap_values = explainer.shap_values(test_x)    
#     shap.summary_plot(shap_values, test_x, feature_names=test_x.columns)
    
    return fs_scores

In [88]:
regressors = [RandomForestRegressor(n_estimators = 200, max_depth = 8), 
              ExtraTreesRegressor(n_estimators = 200, max_depth = 10), 
              XGBRegressor(max_depth=3, n_estimators=120, n_jobs=-1, objectvie='reg:r2', booster='gbtree', random_state=25, learning_rate=0.05, gamma=5), 
              LGBMRegressor(n_estimators=200, max_depth=3, learning_rate=0.05), 
              GradientBoostingRegressor(n_estimators = 150, max_depth = 3, learning_rate=0.05), 
              CatBoostRegressor(learning_rate=0.03, depth=5), 
              AdaBoostRegressor(n_estimators=80, learning_rate=0.05)]


voting = VotingRegressor([('etr', ExtraTreesRegressor(n_estimators = 200, max_depth = 10)), 
                        ('gb', GradientBoostingRegressor(n_estimators=50, max_depth=4)), 
                        ('cat', CatBoostRegressor(learning_rate=0.03, depth=5))], 
                         weights=[0.5, 0.5, 6.0]
                        ) 

In [92]:
res = explainability(voting, 
                    train_dfs[0], test_data)
res

0:	learn: 26.4399176	total: 1.48ms	remaining: 1.48s
1:	learn: 25.9929053	total: 2.67ms	remaining: 1.33s
2:	learn: 25.5786071	total: 3.82ms	remaining: 1.27s
3:	learn: 25.1214411	total: 4.97ms	remaining: 1.24s
4:	learn: 24.7176590	total: 6.11ms	remaining: 1.22s
5:	learn: 24.3163583	total: 7.24ms	remaining: 1.2s
6:	learn: 23.9467923	total: 8.54ms	remaining: 1.21s
7:	learn: 23.5863201	total: 9.73ms	remaining: 1.21s
8:	learn: 23.2187733	total: 10.9ms	remaining: 1.2s
9:	learn: 22.8661114	total: 12ms	remaining: 1.19s
10:	learn: 22.5167448	total: 13.2ms	remaining: 1.19s
11:	learn: 22.1771218	total: 14.3ms	remaining: 1.18s
12:	learn: 21.8530635	total: 15.4ms	remaining: 1.17s
13:	learn: 21.5634146	total: 16.5ms	remaining: 1.16s
14:	learn: 21.3064306	total: 17.7ms	remaining: 1.16s
15:	learn: 21.0415641	total: 18.9ms	remaining: 1.16s
16:	learn: 20.7576228	total: 20ms	remaining: 1.16s
17:	learn: 20.4794232	total: 21.1ms	remaining: 1.15s
18:	learn: 20.2158995	total: 22.3ms	remaining: 1.15s
19:	learn

281:	learn: 10.7473359	total: 322ms	remaining: 819ms
282:	learn: 10.7386375	total: 323ms	remaining: 818ms
283:	learn: 10.7330872	total: 324ms	remaining: 817ms
284:	learn: 10.7194952	total: 325ms	remaining: 816ms
285:	learn: 10.7136313	total: 326ms	remaining: 815ms
286:	learn: 10.7022186	total: 328ms	remaining: 814ms
287:	learn: 10.6975037	total: 329ms	remaining: 812ms
288:	learn: 10.6923328	total: 330ms	remaining: 811ms
289:	learn: 10.6868167	total: 331ms	remaining: 810ms
290:	learn: 10.6790198	total: 332ms	remaining: 808ms
291:	learn: 10.6738387	total: 333ms	remaining: 807ms
292:	learn: 10.6688541	total: 334ms	remaining: 806ms
293:	learn: 10.6597896	total: 335ms	remaining: 805ms
294:	learn: 10.6523523	total: 336ms	remaining: 804ms
295:	learn: 10.6459387	total: 337ms	remaining: 803ms
296:	learn: 10.6416726	total: 339ms	remaining: 801ms
297:	learn: 10.6328219	total: 340ms	remaining: 800ms
298:	learn: 10.6248989	total: 341ms	remaining: 799ms
299:	learn: 10.6166219	total: 342ms	remaining:

579:	learn: 9.0132914	total: 653ms	remaining: 473ms
580:	learn: 9.0102610	total: 654ms	remaining: 472ms
581:	learn: 9.0053918	total: 655ms	remaining: 471ms
582:	learn: 9.0012341	total: 656ms	remaining: 470ms
583:	learn: 8.9985981	total: 658ms	remaining: 468ms
584:	learn: 8.9961008	total: 659ms	remaining: 467ms
585:	learn: 8.9881065	total: 660ms	remaining: 466ms
586:	learn: 8.9841619	total: 661ms	remaining: 465ms
587:	learn: 8.9786274	total: 662ms	remaining: 464ms
588:	learn: 8.9742629	total: 663ms	remaining: 463ms
589:	learn: 8.9711274	total: 665ms	remaining: 462ms
590:	learn: 8.9663510	total: 666ms	remaining: 461ms
591:	learn: 8.9623730	total: 667ms	remaining: 460ms
592:	learn: 8.9560000	total: 668ms	remaining: 459ms
593:	learn: 8.9497063	total: 669ms	remaining: 457ms
594:	learn: 8.9460287	total: 671ms	remaining: 456ms
595:	learn: 8.9429217	total: 672ms	remaining: 455ms
596:	learn: 8.9403121	total: 673ms	remaining: 454ms
597:	learn: 8.9338223	total: 674ms	remaining: 453ms
598:	learn: 

873:	learn: 7.7574154	total: 985ms	remaining: 142ms
874:	learn: 7.7521504	total: 986ms	remaining: 141ms
875:	learn: 7.7481936	total: 988ms	remaining: 140ms
876:	learn: 7.7435405	total: 989ms	remaining: 139ms
877:	learn: 7.7414016	total: 990ms	remaining: 138ms
878:	learn: 7.7365377	total: 991ms	remaining: 136ms
879:	learn: 7.7322520	total: 992ms	remaining: 135ms
880:	learn: 7.7266529	total: 993ms	remaining: 134ms
881:	learn: 7.7214219	total: 994ms	remaining: 133ms
882:	learn: 7.7157378	total: 995ms	remaining: 132ms
883:	learn: 7.7103173	total: 997ms	remaining: 131ms
884:	learn: 7.7090637	total: 998ms	remaining: 130ms
885:	learn: 7.7059161	total: 999ms	remaining: 129ms
886:	learn: 7.7009846	total: 1s	remaining: 127ms
887:	learn: 7.6980697	total: 1s	remaining: 126ms
888:	learn: 7.6925709	total: 1s	remaining: 125ms
889:	learn: 7.6875674	total: 1s	remaining: 124ms
890:	learn: 7.6833439	total: 1s	remaining: 123ms
891:	learn: 7.6786614	total: 1s	remaining: 122ms
892:	learn: 7.6742995	total: 1

Weight,Feature
0.4104 ± 0.0735,BIRTH_WEIGHT
0.1284 ± 0.0228,HEAD_CIRC
0.0600 ± 0.0151,GESTATION
0.0249 ± 0.0085,PRESCRIPTIONS
0.0247 ± 0.0046,sao2_mean
0.0180 ± 0.0152,DATEEVENTS
0.0098 ± 0.0033,WHITE_BLOOD
0.0086 ± 0.0027,D10W_MEAN
0.0071 ± 0.0038,bpCuffDiastolic_mean
0.0063 ± 0.0043,DW10_COUNT
